<a href="https://colab.research.google.com/github/julianayuri/ReconhecimentoPadroes/blob/main/Atividade06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

In [3]:
fri = fetch_openml(name='fri_c2_1000_5', version=2)
X, y = fri.data, fri.target
X.shape, y.shape

((1000, 5), (1000,))

In [4]:
for column in X.columns:
  print(f"{column:>12} {len(set(X[column])):4} {X[column].dtype}")

         oz1 1000 float64
         oz2 1000 float64
         oz3 1000 float64
         oz4 1000 float64
         oz5 1000 float64


In [5]:
from sklearn.model_selection import train_test_split

Xtr, Xte, ytr, yte = train_test_split(X, y, random_state=42)
Xtr.shape, Xte.shape, ytr.shape, yte.shape

((750, 5), (250, 5), (750,), (250,))

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

modelo_voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    #('gnb', GaussianNB()),
    ('per', Perceptron()),
    ('tree', DecisionTreeClassifier())
])


modelo_voting.fit(Xtr, ytr)
vote_pred = modelo_voting.predict(Xte)
vote_hits = vote_pred == yte
vote_hits, sum(vote_hits)/len(vote_hits)

(521    False
 737     True
 740     True
 660     True
 411     True
        ...  
 109     True
 430     True
 77      True
 84      True
 286     True
 Name: binaryClass, Length: 250, dtype: bool, 0.928)

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

parametros = {
    'tree__max_depth': list(range(1,10,2)),
    'knn__n_neighbors': list(range(1,10,2)),
}

modelo_voting_grid = GridSearchCV(modelo_voting, param_grid=parametros)

scores = cross_validate(modelo_voting_grid, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([2.16515446, 2.42689824, 2.01352334, 2.00441766, 2.03533864]),
  'score_time': array([0.0131855 , 0.00863171, 0.00919318, 0.00860834, 0.0088129 ]),
  'test_score': array([0.925, 0.905, 0.92 , 0.87 , 0.88 ])},
 0.9)

In [9]:
modelo_voting_grid.fit(X, y)
modelo_voting_grid.best_estimator_

VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=7)),
                             ('per', Perceptron()),
                             ('tree', DecisionTreeClassifier(max_depth=9))])

In [10]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import StackingClassifier
voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    #('gnb', GaussianNB()),
    ('per', Perceptron()),
    ('tree', DecisionTreeClassifier())
])

modelo_stacking = StackingClassifier([
    ('voting', voting),
    ('extrat', ExtraTreesClassifier()),
    ('ranfor', RandomForestClassifier())
], cv=3, passthrough=True)

modelo_stacking.fit(Xtr, ytr)
stack_pred = modelo_stacking.predict(Xte)
stack_hits = stack_pred == yte
stack_hits, sum(stack_hits)/len(stack_hits)

(521    False
 737     True
 740     True
 660     True
 411     True
        ...  
 109     True
 430     True
 77      True
 84      True
 286     True
 Name: binaryClass, Length: 250, dtype: bool, 0.936)

In [11]:
parametros = {
    'voting__tree__max_depth': list(range(1,10,2)),
    'voting__knn__n_neighbors': list(range(1,10,2))
}

modelo_stacking_grid = GridSearchCV(modelo_stacking, param_grid=parametros)

scores = cross_validate(modelo_stacking_grid, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([128.72280335, 127.1436646 , 127.39743543, 127.99343491,
         129.13150716]),
  'score_time': array([0.03265023, 0.03272748, 0.03208208, 0.03091693, 0.03217363]),
  'test_score': array([0.97 , 0.93 , 0.96 , 0.935, 0.95 ])},
 0.9490000000000001)

In [12]:
modelo_stacking_grid.fit(X, y)
modelo_stacking_grid.best_estimator_

StackingClassifier(cv=3,
                   estimators=[('voting',
                                VotingClassifier(estimators=[('knn',
                                                              KNeighborsClassifier(n_neighbors=1)),
                                                             ('per',
                                                              Perceptron()),
                                                             ('tree',
                                                              DecisionTreeClassifier(max_depth=9))])),
                               ('extrat', ExtraTreesClassifier()),
                               ('ranfor', RandomForestClassifier())],
                   passthrough=True)